# Linear Regression From Scratch

## Mathematical Foundation

Linear regression models the relationship between features **X** and target **y** as:

$$\hat{y} = \mathbf{X}\mathbf{w} + b$$

where:
- $\mathbf{w}$ is the weight vector (learned parameters)
- $b$ is the bias term
- $\hat{y}$ are the predictions

### Loss Function: Mean Squared Error

$$L = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)^2 = \frac{1}{n}||\mathbf{y} - \hat{\mathbf{y}}||^2$$

With L2 regularization (Ridge regression):

$$L = \frac{1}{n}||\mathbf{y} - \hat{\mathbf{y}}||^2 + \frac{\lambda}{2}||\mathbf{w}||^2$$

### Gradient Descent

We minimize the loss by iteratively updating parameters:

$$\mathbf{w} \leftarrow \mathbf{w} - \alpha \frac{\partial L}{\partial \mathbf{w}}$$

$$b \leftarrow b - \alpha \frac{\partial L}{\partial b}$$

where $\alpha$ is the learning rate.

### Gradients

$$\frac{\partial L}{\partial \mathbf{w}} = -\frac{2}{n}\mathbf{X}^T(\mathbf{y} - \hat{\mathbf{y}}) + \lambda\mathbf{w}$$

$$\frac{\partial L}{\partial b} = -\frac{2}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)$$


In [ ]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from linear_regression import LinearRegression

np.random.seed(42)
plt.style.use('seaborn-v0_8-darkgrid')


In [ ]:
# Load preprocessed data
with open('../data_preprocessed.pkl', 'rb') as f:
    data = pickle.load(f)

X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")


## Feature Scaling

Gradient descent converges faster when features are on similar scales. We'll standardize features to have zero mean and unit variance.


In [ ]:
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Feature statistics after scaling:")
print(f"Mean: {X_train_scaled.mean(axis=0)}")
print(f"Std: {X_train_scaled.std(axis=0)}")


## Training Linear Regression Model

Let's train our from-scratch implementation with different hyperparameters to observe convergence behavior.


In [ ]:
# Train model without regularization
model_no_reg = LinearRegression(
    learning_rate=0.01,
    max_iterations=2000,
    regularization=0.0,
    verbose=True
)

model_no_reg.fit(X_train_scaled, y_train)

print(f"\nFinal training loss: {model_no_reg.loss_history[-1]:.6f}")
print(f"Number of iterations: {len(model_no_reg.loss_history)}")


In [ ]:
# Train model with L2 regularization
model_with_reg = LinearRegression(
    learning_rate=0.01,
    max_iterations=2000,
    regularization=0.1,  # L2 regularization strength
    verbose=True
)

model_with_reg.fit(X_train_scaled, y_train)

print(f"\nFinal training loss: {model_with_reg.loss_history[-1]:.6f}")
print(f"Number of iterations: {len(model_with_reg.loss_history)}")


## Visualizing Training Convergence


In [ ]:
# Plot loss convergence
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Full convergence
axes[0].plot(model_no_reg.loss_history, label='No Regularization', linewidth=2)
axes[0].plot(model_with_reg.loss_history, label='L2 Regularization (λ=0.1)', linewidth=2)
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss Convergence')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# First 200 iterations (zoomed in)
axes[1].plot(model_no_reg.loss_history[:200], label='No Regularization', linewidth=2)
axes[1].plot(model_with_reg.loss_history[:200], label='L2 Regularization (λ=0.1)', linewidth=2)
axes[1].set_xlabel('Iteration')
axes[1].set_ylabel('Loss')
axes[1].set_title('Training Loss Convergence (First 200 Iterations)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## Model Evaluation


In [ ]:
# Make predictions
y_pred_no_reg = model_no_reg.predict(X_test_scaled)
y_pred_with_reg = model_with_reg.predict(X_test_scaled)

# Calculate metrics
mse_no_reg = mean_squared_error(y_test, y_pred_no_reg)
mse_with_reg = mean_squared_error(y_test, y_pred_with_reg)

r2_no_reg = model_no_reg.score(X_test_scaled, y_test)
r2_with_reg = model_with_reg.score(X_test_scaled, y_test)

print("Model Performance:")
print("=" * 50)
print(f"No Regularization:")
print(f"  MSE: {mse_no_reg:.6f}")
print(f"  RMSE: {np.sqrt(mse_no_reg):.6f}")
print(f"  R² Score: {r2_no_reg:.6f}")
print(f"\nWith L2 Regularization (λ=0.1):")
print(f"  MSE: {mse_with_reg:.6f}")
print(f"  RMSE: {np.sqrt(mse_with_reg):.6f}")
print(f"  R² Score: {r2_with_reg:.6f}")


In [ ]:
# Visualize predictions vs actual values
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# No regularization
axes[0].scatter(y_test, y_pred_no_reg, alpha=0.5, s=20)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
             'r--', linewidth=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Values')
axes[0].set_ylabel('Predicted Values')
axes[0].set_title(f'Predictions vs Actual (No Regularization)\nR² = {r2_no_reg:.4f}')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# With regularization
axes[1].scatter(y_test, y_pred_with_reg, alpha=0.5, s=20)
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
             'r--', linewidth=2, label='Perfect Prediction')
axes[1].set_xlabel('Actual Values')
axes[1].set_ylabel('Predicted Values')
axes[1].set_title(f'Predictions vs Actual (L2 Regularization)\nR² = {r2_with_reg:.4f}')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## Learning Rate Experiment

Let's observe how different learning rates affect convergence.


In [ ]:
# Test different learning rates
learning_rates = [0.001, 0.01, 0.1, 0.5]
models_lr = {}

for lr in learning_rates:
    model = LinearRegression(
        learning_rate=lr,
        max_iterations=1000,
        regularization=0.0,
        verbose=False
    )
    model.fit(X_train_scaled, y_train)
    models_lr[lr] = model
    print(f"LR={lr}: Final loss = {model.loss_history[-1]:.6f}, Iterations = {len(model.loss_history)}")


In [ ]:
# Plot convergence for different learning rates
plt.figure(figsize=(12, 6))

for lr in learning_rates:
    plt.plot(models_lr[lr].loss_history, label=f'LR = {lr}', linewidth=2)

plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Effect of Learning Rate on Convergence')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')  # Log scale for better visualization
plt.tight_layout()
plt.show()


## Key Observations

1. **Convergence**: Gradient descent successfully minimizes the loss function
2. **Regularization**: L2 regularization can help prevent overfitting (though effect may be subtle on this dataset)
3. **Learning Rate**: Too small → slow convergence, too large → instability/divergence
4. **Vectorization**: NumPy operations enable efficient computation on large datasets

The implementation demonstrates core ML principles: optimization, regularization, and hyperparameter sensitivity.
